In [1]:
# Import modules
import os
import sys
sys.path.append(r'..\Hardware_src') # Make the change according to your path

# Python modules for figure visualization
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy import io
from scipy import signal
import math
import cv2
import screeninfo
from scipy.linalg import hadamard    
from scipy.optimize import minimize
from pathlib import Path

## 1D Identity Reconstruction

In [24]:
# Load data
file_path = Path(os.getcwd())
data_path = file_path.joinpath('data')

black_subtract_1d = np.loadtxt(data_path.joinpath('black_subtract_1d_59.csv'), delimiter=',')
black_subtract_1d = black_subtract_1d.reshape(1024,1280,16)
mask1d = np.loadtxt(data_path.joinpath('mask_1d_59.csv'), delimiter=',')
mask1d = mask1d.reshape(16,16,16)
scene_1d = np.loadtxt(data_path.joinpath('scene_1d_59.csv'), delimiter=',')
scene_1d = scene_1d.reshape(1024,1280,16)

In [ ]:
# View images

In [ ]:
# Compressed Sensing Alg 1
# Solve y = A*x for x using L1 minimum norm solution
def L1_norm(x):
    return np.linal.norm(x, ord=1)

constr = ({'type': 'eq', 'fun': lambda x: A @ x - y})
x0 = np.linalg.pinv(A) @ y # initialize minimization with L2 solution
res = minimize(L1_norm, x0, method='SLSQP', constraints=constr)
x_L1 = res.x

plt.imshow(x_L1)
plt.colorbar()
plt.show()